# База, обработка данных


In [24]:
import getpass

In [25]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.preprocessing import StandardScaler

In [26]:
#BINANCE_API_KEY = getpass.getpass('BINANCE_API_KEY: ')
#BINANCE_SECRET_KEY = getpass.getpass('BINANCE_SECRET_KEY: ')

Важное уточнение, почему ниже все закомменчено:
Binance не дает возможности качать данные по API из любого места, именно поэтому коллаб иногда может их скачать, а иногда - нет. Из-за этого в данном файле оно закомменчено, в итоговом сервисе с российским IP будет функционировать полноценно

In [27]:
import ccxt
import os
import time
from datetime import datetime
from dotenv import load_dotenv
'''
load_dotenv()

binance = ccxt.binance({
    'apiKey': BINANCE_API_KEY ,
    'secret': BINANCE_SECRET_KEY,
    'enableRateLimit': True
})

def compute_indicators(df):
    # EMA
    df['EMA_10'] = df['close'].ewm(span=10, adjust=False).mean()
    df['EMA_50'] = df['close'].ewm(span=50, adjust=False).mean()

    # RSI
    delta = df['close'].diff()
    gain = np.where(delta > 0, delta, 0)
    loss = np.where(delta < 0, -delta, 0)
    avg_gain = pd.Series(gain).rolling(window=14).mean()
    avg_loss = pd.Series(loss).rolling(window=14).mean()
    rs = avg_gain / avg_loss
    df['RSI_14'] = 100 - (100 / (1 + rs))

    # MACD
    ema12 = df['close'].ewm(span=12, adjust=False).mean()
    ema26 = df['close'].ewm(span=26, adjust=False).mean()
    df['MACD'] = ema12 - ema26
    df['MACD_signal'] = df['MACD'].ewm(span=9, adjust=False).mean()

    # Bollinger Bands
    sma20 = df['close'].rolling(window=20).mean()
    std20 = df['close'].rolling(window=20).std()
    df['Bollinger_Upper'] = sma20 + (2 * std20)
    df['Bollinger_Lower'] = sma20 - (2 * std20)

    # ATR
    high_low = df['high'] - df['low']
    high_close = np.abs(df['high'] - df['close'].shift())
    low_close = np.abs(df['low'] - df['close'].shift())
    tr = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
    df['ATR_14'] = tr.rolling(window=14).mean()

    return df

# Пошаговая загрузка за 5 лет
def fetch_full_history(symbol, timeframe='1h', since=None, max_batches=1000, batch_limit=1500, sleep_sec=1):
    all_ohlcv = []
    since = since or binance.parse8601('2019-01-01T00:00:00Z')

    for i in range(max_batches):
        try:
            print(f"[{symbol}] Батч {i + 1}, since = {datetime.utcfromtimestamp(since / 1000)}")
            ohlcv = binance.fetch_ohlcv(symbol, timeframe=timeframe, since=since, limit=batch_limit)
            if not ohlcv:
                break
            all_ohlcv.extend(ohlcv)
            since = ohlcv[-1][0] + 1
            time.sleep(sleep_sec)
        except Exception as e:
            print(f"Ошибка: {e}")
            break

    df = pd.DataFrame(all_ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    return df

# Обработка всех монет
def fetch_all_symbols_full(symbols, timeframe='1h', max_batches=1000):
    result = {}
    for symbol in symbols:
        try:
            df = fetch_full_history(symbol, timeframe=timeframe, max_batches=max_batches)
            df = compute_indicators(df)
            result[symbol] = df
        except Exception as e:
            print(f"Ошибка при {symbol}: {e}")
    return result '''

'\nload_dotenv()\n\nbinance = ccxt.binance({\n    \'apiKey\': BINANCE_API_KEY ,\n    \'secret\': BINANCE_SECRET_KEY,\n    \'enableRateLimit\': True\n})\n\ndef compute_indicators(df):\n    # EMA\n    df[\'EMA_10\'] = df[\'close\'].ewm(span=10, adjust=False).mean()\n    df[\'EMA_50\'] = df[\'close\'].ewm(span=50, adjust=False).mean()\n\n    # RSI\n    delta = df[\'close\'].diff()\n    gain = np.where(delta > 0, delta, 0)\n    loss = np.where(delta < 0, -delta, 0)\n    avg_gain = pd.Series(gain).rolling(window=14).mean()\n    avg_loss = pd.Series(loss).rolling(window=14).mean()\n    rs = avg_gain / avg_loss\n    df[\'RSI_14\'] = 100 - (100 / (1 + rs))\n\n    # MACD\n    ema12 = df[\'close\'].ewm(span=12, adjust=False).mean()\n    ema26 = df[\'close\'].ewm(span=26, adjust=False).mean()\n    df[\'MACD\'] = ema12 - ema26\n    df[\'MACD_signal\'] = df[\'MACD\'].ewm(span=9, adjust=False).mean()\n\n    # Bollinger Bands\n    sma20 = df[\'close\'].rolling(window=20).mean()\n    std20 = df[\'clos

In [28]:
symbols = ['BTC/USDT', 'ETH/USDT', 'SOL/USDT', 'XRP/USDT', 'TRX/USDT']
#data = fetch_all_symbols_full(symbols, timeframe='1h', max_batches=70)  # 35 батчей по 1500 = ~52 000 часов ≈ 6 лет

In [29]:
#btc_df.timestamp.min()

In [30]:
def load_all_from_csv(folder='crypto_csv'):
    data_dict = {}
    for fname in os.listdir(folder):
        if fname.endswith(".csv"):
            symbol = fname.replace(".csv", "").replace("_", "/")
            df = pd.read_csv(os.path.join(folder, fname), parse_dates=['timestamp'])
            data_dict[symbol] = df
    return data_dict

In [31]:
data = load_all_from_csv('crypto_csv_hourly')

In [33]:
data['BTC/USDT'].columns

Index(['timestamp', 'open', 'high', 'low', 'close', 'volume', 'EMA_10',
       'EMA_50', 'RSI_14', 'MACD', 'MACD_signal', 'Bollinger_Upper',
       'Bollinger_Lower', 'ATR_14'],
      dtype='object')

In [10]:
def add_cci(df, window=20):
    tp = (df['high'] + df['low'] + df['close']) / 3
    ma = tp.rolling(window).mean()
    md = tp.rolling(window).apply(lambda x: np.mean(np.abs(x - np.mean(x))), raw=True)
    df['cci'] = (tp - ma) / (0.015 * (md + 1e-8))
    return df

# === Препроцессинг ===
def standardize_features(df, feature_cols):
    df = df.copy()
    scaler = StandardScaler()
    df[feature_cols] = scaler.fit_transform(df[feature_cols].fillna(0))
    return df

def prepare_dataframe(df, feature_cols):
    df = df.copy()
    df = add_cci(df)
    df = df.dropna()
    df = standardize_features(df, feature_cols)
    return df, feature_cols

def create_multistep_dataset(df, feature_cols, target_col='close', window_size=96, horizon=48):
    X, y = [], []
    for i in range(len(df) - window_size - horizon):
        window = df.iloc[i:i + window_size]
        target_seq = df.iloc[i + window_size:i + window_size + horizon][target_col].values
        if window[feature_cols].isnull().any().any() or np.isnan(target_seq).any():
            continue
        X.append(window[feature_cols].values)
        y.append(target_seq)
    return np.array(X), np.array(y)

In [11]:
feature_cols = ['open', 'high', 'low', 'volume', 'EMA_10',
       'EMA_50', 'RSI_14', 'MACD', 'MACD_signal', 'Bollinger_Upper',
       'Bollinger_Lower', 'ATR_14']

In [12]:
def create_multistep_dataset_delta(df, feature_cols, target_col='close', window_size=168, horizon=48):
    X, y, start_prices = [], [], []
    close = df[target_col].values
    for i in range(len(df) - window_size - horizon):
        window = df.iloc[i:i + window_size]
        start_price = close[i + window_size - 1]
        future_price = close[i + window_size:i + window_size + horizon]
        delta = future_price - start_price
        if window[feature_cols].isnull().any().any() or np.isnan(delta).any():
            continue
        X.append(window[feature_cols].values)
        y.append(delta)
        start_prices.append(start_price)
    return np.array(X), np.array(y), np.array(start_prices)


In [13]:
#X, y, coin_ids = build_full_dataset(data, feature_cols, target_col='close', window_size=24, horizon=48)

#print("Готово! X.shape =", X.shape, ", y.shape =", y.shape)

# Версия моделей и обработки данных от 14.05

In [ ]:
def prepare_dataframe_for_delta(df, window_size=96, horizon=48):
    df = df.copy()
    df['tp'] = (df['high'] + df['low'] + df['close']) / 3
    df['cci'] = (df['tp'] - df['tp'].rolling(20).mean()) / (0.015 * df['tp'].rolling(20).std() + 1e-8)
    df = df.dropna()

    feature_cols = ['open', 'high', 'low', 'volume', 'rsi', 'macd', 'ema_10', 'obv', 'cci']
    scaler = StandardScaler()
    df[feature_cols] = scaler.fit_transform(df[feature_cols])

    return df, feature_cols, scaler

# === Создание окон с дельтой ===
def create_delta_price_dataset(df, feature_cols, target_col='close', window_size=96, horizon=48):
    X, y, start_prices = [], [], []
    close = df[target_col].values
    for i in range(len(df) - window_size - horizon):
        window = df.iloc[i:i + window_size]
        start_price = close[i + window_size - 1]
        end_price = close[i + window_size + horizon - 1]
        delta = end_price - start_price
        if window[feature_cols].isnull().any().any() or np.isnan(delta):
            continue
        X.append(window[feature_cols].values)
        y.append(delta)
        start_prices.append(start_price)
    return np.array(X), np.array(y), np.array(start_prices)

In [ ]:
# === Performer block ===
class SimplePerformerBlock(nn.Module):
    def __init__(self, dim, heads=4, dropout=0.1):
        super().__init__()
        self.qkv = nn.Linear(dim, dim * 3)
        self.out_proj = nn.Linear(dim, dim)
        self.heads = heads
        self.dropout = nn.Dropout(dropout)
        self.scale = (dim // heads) ** -0.5

    def forward(self, x):
        B, T, D = x.shape
        H = self.heads
        qkv = self.qkv(x)
        q, k, v = qkv.chunk(3, dim=-1)
        q = q.view(B, T, H, D // H).transpose(1, 2)
        k = k.view(B, T, H, D // H).transpose(1, 2)
        v = v.view(B, T, H, D // H).transpose(1, 2)
        attn = F.softmax((q @ k.transpose(-2, -1)) * self.scale, dim=-1)
        out = attn @ v
        out = out.transpose(1, 2).contiguous().view(B, T, D)
        return self.out_proj(out)

# === Гибридная модель Performer + BiLSTM ===
class HybridDeltaModel(nn.Module):
    def __init__(self, input_dim, hidden_dim=128):
        super().__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True, bidirectional=True)
        self.performer_block = SimplePerformerBlock(dim=input_dim)
        self.fusion = nn.Sequential(
            nn.Linear(hidden_dim * 2 + input_dim, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 1)
        )

    def forward(self, x):
        _, (h_n, _) = self.lstm(x)
        h_bilstm = torch.cat((h_n[-2], h_n[-1]), dim=-1)  # (B, 2*hidden_dim)
        h_perf = self.performer_block(x).mean(dim=1)  # mean pooling по времени
        h = torch.cat([h_bilstm, h_perf], dim=-1)
        return self.fusion(h).squeeze()



In [ ]:
def train_delta_model(X, y, input_dim, epochs=10, batch_size=32, lr=1e-3):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = DeltaPredictor(input_dim=input_dim).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()

    X_train = torch.tensor(X, dtype=torch.float32)
    y_train = torch.tensor(y, dtype=torch.float32)

    loader = DataLoader(TensorDataset(X_train, y_train), batch_size=batch_size, shuffle=True)

    for epoch in range(epochs):
        model.train()
        epoch_loss = 0.0
        for xb, yb in loader:
            xb, yb = xb.to(device), yb.to(device)
            optimizer.zero_grad()
            preds = model(xb)
            loss = criterion(preds, yb)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        print(f"Epoch {epoch+1}/{epochs} — Loss: {epoch_loss/len(loader):.4f}")

    return model

In [ ]:
df, feature_cols, scaler = prepare_dataframe_for_delta(df)
X, y, start_prices = create_delta_price_dataset(df, feature_cols)

In [ ]:
all_X, all_y, all_start_prices = [], [], []
all_scalers = {}
feature_cols = None

In [ ]:
for symbol, df in data_dict.items():
    print(f"🔧 Подготовка данных для {symbol}")
    df_prepared, features, scaler = prepare_dataframe_for_delta(df, window_size, horizon)
    feature_cols = features  # для совместимости
    X, y, starts = create_delta_price_dataset(df_prepared, feature_cols, window_size=window_size, horizon=horizon)
    if len(X) == 0:
        continue
    all_X.append(X)
    all_y.append(y)
    all_start_prices.append(starts)
    all_scalers[symbol] = scaler

In [ ]:
X_all = np.concatenate(all_X, axis=0)
y_all = np.concatenate(all_y, axis=0)
start_all = np.concatenate(all_start_prices, axis=0)

In [ ]:
X_train, X_test, y_train, y_test, start_train, start_test = train_test_split(
        X_all, y_all, start_all, test_size=0.1, random_state=42
    )

In [ ]:
train_loader = DataLoader(
        TensorDataset(torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.float32)),
        batch_size=batch_size, shuffle=True
    )

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = HybridDeltaModel(input_dim=X_all.shape[-1]).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
criterion = torch.nn.MSELoss()

In [ ]:
for epoch in range(10):
    model.train()
    epoch_loss = 0
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        preds = model(xb)
        loss = criterion(preds, yb)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    print(f"Epoch {epoch+1}/{10} — Train Loss: {epoch_loss / len(train_loader):.4f}")


In [ ]:
# === Тестовый блок с восстановлением цен ===
def test_model(model, X_test, y_test, start_prices):
    model.eval()
    device = next(model.parameters()).device
    loader = DataLoader(TensorDataset(torch.tensor(X_test, dtype=torch.float32),
                                      torch.tensor(y_test, dtype=torch.float32)),
                        batch_size=32)

    all_preds = []
    with torch.no_grad():
        for xb, _ in loader:
            xb = xb.to(device)
            preds = model(xb).cpu().numpy()
            all_preds.extend(preds)

    y_pred = np.array(all_preds)
    y_true = y_test
    predicted_prices = start_prices + y_pred
    actual_prices = start_prices + y_true

    returns = predicted_prices - start_prices
    true_returns = actual_prices - start_prices
    direction_match = np.sign(returns) == np.sign(true_returns)

    da = np.mean(direction_match)
    mean_return = returns.mean()
    sharpe = mean_return / (returns.std() + 1e-8)
    equity = np.cumsum(returns)
    max_drawdown = np.max(np.maximum.accumulate(equity) - equity)

    print(f"Directional Accuracy: {da:.3f}")
    print(f"Mean Return: {mean_return:.2f}")
    print(f"Sharpe Ratio: {sharpe:.2f}")
    print(f"Max Drawdown: {max_drawdown:.2f}")

    return {
        "DA": da,
        "Mean Return": mean_return,
        "Sharpe": sharpe,
        "MDD": max_drawdown
    }


In [ ]:
test_model(model, X_test, y_test, start_test)

# Модели и обработка данных под конкретный эксперимент(пока что единственный рабочий)


In [14]:
class BiLSTMTemporalModel(nn.Module):
    def __init__(self, input_dim, hidden_dim=128, num_layers=2, dropout=0.1, output_dim=128):
        super().__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers=num_layers, batch_first=True,
                            bidirectional=True, dropout=dropout if num_layers > 1 else 0.0)
        self.output_proj = nn.Linear(hidden_dim * 2, output_dim)

    def forward(self, x):
        _, (h_n, _) = self.lstm(x)
        h_forward, h_backward = h_n[-2], h_n[-1]
        h_combined = torch.cat([h_forward, h_backward], dim=-1)
        return self.output_proj(h_combined)


In [15]:
class SimplePerformerBlock(nn.Module):
    def __init__(self, dim, heads=4, dropout=0.1):
        super().__init__()
        self.qkv = nn.Linear(dim, dim * 3)
        self.out_proj = nn.Linear(dim, dim)
        self.heads = heads
        self.dropout = nn.Dropout(dropout)
        self.scale = (dim // heads) ** -0.5

    def forward(self, x):
        B, T, D = x.shape
        H = self.heads
        qkv = self.qkv(x)
        q, k, v = qkv.chunk(3, dim=-1)
        q = q.view(B, T, H, D // H).transpose(1, 2)
        k = k.view(B, T, H, D // H).transpose(1, 2)
        v = v.view(B, T, H, D // H).transpose(1, 2)
        attn_weights = F.softmax((q @ k.transpose(-2, -1)) * self.scale, dim=-1)
        attn_output = attn_weights @ v
        attn_output = attn_output.transpose(1, 2).contiguous().view(B, T, D)
        return self.out_proj(attn_output)


class PerformerTemporalModel(nn.Module):
    def __init__(self, input_dim, hidden_dim=128, heads=4, depth=2, dropout=0.1, output_dim=128):
        super().__init__()
        self.input_proj = nn.Linear(input_dim, hidden_dim)
        self.blocks = nn.ModuleList([
            nn.Sequential(
                SimplePerformerBlock(hidden_dim, heads, dropout),
                nn.LayerNorm(hidden_dim),
                nn.Sequential(
                    nn.Linear(hidden_dim, hidden_dim * 4),
                    nn.GELU(),
                    nn.Dropout(dropout),
                    nn.Linear(hidden_dim * 4, hidden_dim)
                ),
                nn.LayerNorm(hidden_dim)
            )
            for _ in range(depth)
        ])
        self.cls_token = nn.Parameter(torch.randn(1, 1, hidden_dim))
        self.output_proj = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        B, T, _ = x.shape
        x = self.input_proj(x)
        cls_tokens = self.cls_token.expand(B, -1, -1)
        x = torch.cat((cls_tokens, x), dim=1)
        for block in self.blocks:
            x = x + block(x)
        cls_output = x[:, 0, :]
        return self.output_proj(cls_output)


In [16]:
class MultiStepHybridModel(nn.Module):
    def __init__(self, input_dim, temporal_dim=128, fusion_dim=256, output_horizon=48):
        super().__init__()
        self.bilstm = BiLSTMTemporalModel(input_dim, hidden_dim=temporal_dim, output_dim=temporal_dim)
        self.performer = PerformerTemporalModel(input_dim, hidden_dim=temporal_dim, output_dim=temporal_dim)
        self.fusion = nn.Sequential(
            nn.Linear(temporal_dim * 2, fusion_dim),
            nn.ReLU(),
            nn.Linear(fusion_dim, output_horizon)
        )

    def forward(self, x):
        h_bilstm = self.bilstm(x)
        h_perf = self.performer(x)
        h = torch.cat([h_bilstm, h_perf], dim=-1)
        return self.fusion(h)

In [17]:
def make_dataloader(X, y, coin_ids, batch_size=32, shuffle=True):
    X_tensor = torch.tensor(X, dtype=torch.float32)
    y_tensor = torch.tensor(y, dtype=torch.float32)
    id_tensor = torch.tensor(coin_ids, dtype=torch.long)
    dataset = torch.utils.data.TensorDataset(X_tensor, y_tensor, id_tensor)
    return torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)

In [18]:
split_data = {}

for symbol, df in data.items():
    df_prepared, feature_cols = prepare_dataframe(df, feature_cols)
    X, y, start_prices = create_multistep_dataset_delta(df_prepared, feature_cols)

    if len(X) == 0:
        continue

    n = len(X)
    train_end = int(n * 0.75)
    val_end = int(n * 0.90)

    split_data[symbol] = {
        "X_train": X[:train_end],
        "y_train": y[:train_end],
        "X_val": X[train_end:val_end],
        "y_val": y[train_end:val_end],
        "X_test": X[val_end:],
        "y_test": y[val_end:],
        "start_prices": start_prices[val_end:],  # только тестовые стартовые цены
        "feature_cols": feature_cols
    }

In [19]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model = MultiStepHybridModel(input_dim=len(feature_cols)).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.MSELoss()

cuda


In [20]:


for symbol, data in split_data.items():
    print(f"\n--- Обучение на монете {symbol} ---")

    # Даталоадеры
    def get_loader(x, y):
        ds = torch.utils.data.TensorDataset(torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.float32))
        return torch.utils.data.DataLoader(ds, batch_size=32, shuffle=True)

    train_loader = get_loader(data['X_train'], data['y_train'])
    val_loader = get_loader(data['X_val'], data['y_val'])

    for epoch in range(10):
        model.train()
        train_loss = 0
        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)
            optimizer.zero_grad()
            preds = model(xb)
            loss = criterion(preds, yb)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        # Валидация
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for xb, yb in val_loader:
                xb, yb = xb.to(device), yb.to(device)
                val_loss += criterion(model(xb), yb).item()

        print(f"{symbol} — Epoch {epoch+1}: Train Loss={train_loss/len(train_loader):.6f}, Val Loss={val_loss/len(val_loader):.6f}")


--- Обучение на монете BTC/USDT ---
BTC/USDT — Epoch 1: Train Loss=1125264.141431, Val Loss=2324771.358654
BTC/USDT — Epoch 2: Train Loss=1103847.727194, Val Loss=2409590.117548
BTC/USDT — Epoch 3: Train Loss=1095179.058759, Val Loss=2445786.024279
BTC/USDT — Epoch 4: Train Loss=1076358.822436, Val Loss=2477835.131731
BTC/USDT — Epoch 5: Train Loss=1068564.566578, Val Loss=2632882.278365
BTC/USDT — Epoch 6: Train Loss=1053053.059962, Val Loss=2494401.866827
BTC/USDT — Epoch 7: Train Loss=1005188.678827, Val Loss=2846999.941827
BTC/USDT — Epoch 8: Train Loss=954948.166444, Val Loss=3056637.936058
BTC/USDT — Epoch 9: Train Loss=907884.460667, Val Loss=3149393.535096
BTC/USDT — Epoch 10: Train Loss=878660.124687, Val Loss=3496322.039904

--- Обучение на монете ETH/USDT ---
ETH/USDT — Epoch 1: Train Loss=21538.821582, Val Loss=23578.647908
ETH/USDT — Epoch 2: Train Loss=7587.255018, Val Loss=16117.168883
ETH/USDT — Epoch 3: Train Loss=6474.592466, Val Loss=14045.897314
ETH/USDT — Epoch 4:

In [22]:
results = {}

for symbol, data in split_data.items():
    print(f"\n=== Тест на {symbol} ===")
    X_test, y_test, start_prices = data['X_test'], data['y_test'], data['start_prices']

    test_loader = torch.utils.data.DataLoader(
        torch.utils.data.TensorDataset(
            torch.tensor(X_test, dtype=torch.float32),
            torch.tensor(y_test, dtype=torch.float32)
        ),
        batch_size=32
    )

    all_preds, all_truths = [], []
    model.eval()
    with torch.no_grad():
        for xb, yb in test_loader:
            xb = xb.to(device)
            preds = model(xb).cpu().numpy()
            all_preds.append(preds)
            all_truths.append(yb.numpy())

    y_pred = np.vstack(all_preds)
    y_true = np.vstack(all_truths)

    # Восстановление предсказанных и реальных цен
    pred_price = start_prices + y_pred[:, -1]
    true_price = start_prices + y_true[:, -1]

    returns = pred_price - start_prices
    real_returns = true_price - start_prices
    correct_direction = np.sign(returns) == np.sign(real_returns)

    da = np.mean(correct_direction)
    hit = np.mean((np.abs(returns) > 0) & correct_direction)
    mean_ret = returns.mean()
    std_ret = returns.std()
    downside = np.std(returns[returns < 0]) if np.any(returns < 0) else 1.0
    sharpe = mean_ret / (std_ret + 1e-8)
    sortino = mean_ret / (downside + 1e-8)

    equity = np.cumsum(returns)
    mdd = np.max(np.maximum.accumulate(equity) - equity)

    results[symbol] = {
        "DA": da, "Hit": hit, "Mean Return": mean_ret,
        "Sharpe": sharpe, "Sortino": sortino, "MDD": mdd
    }

# Печать итогов
print("\n📊 Метрики по монетам:")
for sym, r in results.items():
    print(f"{sym:10} | DA: {r['DA']:.3f} | Hit: {r['Hit']:.3f} | Return: {r['Mean Return']:.2f} | Sharpe: {r['Sharpe']:.2f} | Sortino: {r['Sortino']:.2f} | MDD: {r['MDD']:.2f}")



=== Тест на BTC/USDT ===

=== Тест на ETH/USDT ===

=== Тест на TRX/USDT ===

=== Тест на XRP/USDT ===

=== Тест на SOL/USDT ===

📊 Метрики по монетам:
BTC/USDT   | DA: 0.479 | Hit: 0.479 | Return: -8.88 | Sharpe: -0.51 | Sortino: -0.84 | MDD: 50330.96
ETH/USDT   | DA: 0.502 | Hit: 0.502 | Return: -3.56 | Sharpe: -0.23 | Sortino: -0.26 | MDD: 23657.51
TRX/USDT   | DA: 0.468 | Hit: 0.468 | Return: -2.61 | Sharpe: -0.09 | Sortino: -0.14 | MDD: 32955.37
XRP/USDT   | DA: 0.514 | Hit: 0.514 | Return: 4.69 | Sharpe: 0.18 | Sortino: 0.40 | MDD: 12654.66
SOL/USDT   | DA: 0.522 | Hit: 0.522 | Return: -8.76 | Sharpe: -0.51 | Sortino: -0.58 | MDD: 36176.71


# Trash

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MultiStepHybridModel(input_dim=7).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.MSELoss()

for symbol, df in data.items():
    print(f"\n--- Обработка монеты: {symbol} ---")
    df_prepared, feature_cols = prepare_dataframe(df)
    X, y = create_multistep_dataset(df_prepared, feature_cols)

    if len(X) == 0:
        print("Пропущено: недостаточно данных.")
        continue

    dataset = torch.utils.data.TensorDataset(torch.tensor(X, dtype=torch.float32), torch.tensor(y, dtype=torch.float32))
    loader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)

    for epoch in range(30):
        model.train()
        total_loss = 0
        for xb, yb in loader:
            xb, yb = xb.to(device), yb.to(device)
            optimizer.zero_grad()
            preds = model(xb)
            loss = criterion(preds, yb)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"{symbol} — Epoch {epoch+1}: Loss = {total_loss/len(loader):.6f}")


In [ ]:
from sklearn.model_selection import train_test_split

# === 1. Разбиваем выборку ===
X_train_full, X_test, y_train_full, y_test, id_train_full, id_test = train_test_split(
    X, y, coin_ids, test_size=0.1, random_state=42
)

X_train, X_val, y_train, y_val, id_train, id_val = train_test_split(
    X_train_full, y_train_full, id_train_full, test_size=0.2, random_state=42
)

# === 2. Создаём DataLoader'ы ===
train_loader = make_dataloader(X_train, y_train, id_train, batch_size=32, shuffle=False)
val_loader = make_dataloader(X_val, y_val, id_val, batch_size=32, shuffle=False)
test_loader = make_dataloader(X_test, y_test, id_test, batch_size=32, shuffle=False)

# === 3. Объявляем модель ===
num_coins = len(set(coin_ids))  # уникальных монет
input_dim = X.shape[-1]
horizon = y.shape[-1]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

model = MultiStepHybridModel(
    input_dim=input_dim,
    num_coins=num_coins,
    output_horizon=horizon
).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.MSELoss()

# === 4. Обучение ===
num_epochs = 18
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for xb, yb, coin_id in train_loader:
        xb, yb, coin_id = xb.to(device), yb.to(device), coin_id.to(device)
        optimizer.zero_grad()
        preds = model(xb, coin_id)
        loss = criterion(preds, yb)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    val_loss = 0
    model.eval()
    with torch.no_grad():
        for xb, yb, coin_id in val_loader:
            xb, yb, coin_id = xb.to(device), yb.to(device), coin_id.to(device)
            preds = model(xb, coin_id)
            val_loss += criterion(preds, yb).item()

    print(f"Epoch {epoch+1}/{num_epochs} — Train Loss: {total_loss/len(train_loader):.6f}, Val Loss: {val_loss/len(val_loader):.6f}")


cuda
Epoch 1/18 — Train Loss: 58925220.801969, Val Loss: 34120494.982131
Epoch 2/18 — Train Loss: 13407841.820598, Val Loss: 1194914.618652
Epoch 3/18 — Train Loss: 2143704.108968, Val Loss: 743696.980838
Epoch 4/18 — Train Loss: 1028531.441911, Val Loss: 628343.920961
Epoch 5/18 — Train Loss: 923688.433078, Val Loss: 691504.022469
Epoch 6/18 — Train Loss: 864325.147131, Val Loss: 538008.668185
Epoch 7/18 — Train Loss: 774295.645730, Val Loss: 577735.926757
Epoch 8/18 — Train Loss: 1532728.193121, Val Loss: 8479673.303228
Epoch 9/18 — Train Loss: 797430.474596, Val Loss: 556947.170496
Epoch 10/18 — Train Loss: 742038.252574, Val Loss: 659708.979397
Epoch 11/18 — Train Loss: 28704460.651206, Val Loss: 33412117.274444
Epoch 12/18 — Train Loss: 37709343.624332, Val Loss: 31424165.814038
Epoch 13/18 — Train Loss: 29235009.416161, Val Loss: 15730771.954231
Epoch 14/18 — Train Loss: 32303415.410448, Val Loss: 2859434.005984
Epoch 15/18 — Train Loss: 14006464.008978, Val Loss: 1800245.593876


In [ ]:
def evaluate_model(y_true, y_pred, horizon_step=47):
    pred = y_pred[:, horizon_step]
    real = y_true[:, horizon_step]
    start_price = y_true[:, 0]

    real_direction = np.sign(real - start_price)
    pred_direction = np.sign(pred - start_price)
    directional_accuracy = np.mean(real_direction == pred_direction)

    hits = (pred > start_price) & (real > start_price)
    hit_ratio = hits.sum() / ((pred > start_price).sum() + 1e-8)

    model_returns = np.where(pred > start_price, real - start_price, 0)

    mean_return = np.mean(model_returns)
    std_return = np.std(model_returns)
    sharpe_ratio = mean_return / (std_return + 1e-8)

    downside_std = np.std(model_returns[model_returns < 0]) + 1e-8
    sortino_ratio = mean_return / downside_std

    cumulative = np.cumsum(model_returns)
    peak = np.maximum.accumulate(cumulative)
    drawdown = peak - cumulative
    max_drawdown = np.max(drawdown)

    return {
        "Directional Accuracy": directional_accuracy,
        "Hit Ratio": hit_ratio,
        "Mean Return ($)": mean_return,
        "Sharpe Ratio": sharpe_ratio,
        "Sortino Ratio": sortino_ratio,
        "Max Drawdown ($)": max_drawdown
    }

In [ ]:
model.eval()
all_preds, all_truths = [], []
with torch.no_grad():
    for xb, yb, coin_id in test_loader:
        xb = xb.to(device)
        coin_id = coin_id.to(device)
        preds = model(xb, coin_id).cpu().numpy()
        all_preds.append(preds)
        all_truths.append(yb.numpy())

y_pred_test = np.vstack(all_preds)
y_true_test = np.vstack(all_truths)

# Расчет метрик
metrics_result = evaluate_model(y_true_test, y_pred_test, horizon_step=47)
metrics_result

{'Directional Accuracy': np.float64(0.4982740962712893),
 'Hit Ratio': np.float64(0.536305147058002),
 'Mean Return ($)': np.float32(11.374677),
 'Sharpe Ratio': np.float32(0.036527734),
 'Sortino Ratio': np.float32(0.023196382),
 'Max Drawdown ($)': np.float32(29290.469)}